In [105]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Conv2DTranspose, Flatten
from tensorflow.keras.layers import Concatenate, ZeroPadding2D, PReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
print(tf.__version__)

2.3.0


In [135]:
class LayerUtil:
    def __init__(self):
        pass
    
    def init_block(self, inputs, name):
        zero_pad = ZeroPadding2D(1)(inputs)
        conv_layer = Conv2D(filters = 13, kernel_size = (3,3), strides = 2)(zero_pad)
        pool_layer = MaxPool2D(pool_size = (2,2), strides = 2)(inputs)
        return Concatenate(axis = 3,name = name)([conv_layer, pool_layer])
    
    
    ##Each bottleneck consist of:
    ##Three convolutional layers:
        #1x1 regular convolution (for dimensionality reduction)
        #Main conv layer either a regular, dilated, of deconv with 3x3 filters
        
    def bn_prelu(self, inputs):
        batch_norm = BatchNormalization()(inputs)
        prelu = PReLU()(batch_norm)
        return prelu
        
    def bottleneck_downsample(self, inputs, down_filter, ):
        print(inputs.shape)
        dim_reduction = Conv2D(filters = filters, kernel_size = (2,2), strides = (2,2), padding = 'same')(inputs)
        print(dim_reduction.shape)
        prelu1 = self.bn_prelu(dim_reduction)
        
        max_pool = MaxPool2D(pool_size = 2, strides = (2,2))(inputs)
#         print(max_pool.shape)
        
        ##assuming downsampling conv is a regular convolution???
        main_conv = Conv2D(filters = filters, kernel_size = (3,3), padding = 'same')(prelu1)
        prelu2 = self.bn_prelu(main_conv)
        
        dim_expansion = Conv2D(filters = filters, kernel_size = (1,1), padding = 'same')(prelu2)
        prelu3 = self.bn_prelu(dim_expansion)
        
        
        
        return prelu2

class ENet:
    def __init__(self):
        self.helper = LayerUtil()
    
    def build_model(self):
        inputs = Input(shape=(512, 512, 3))
        concat = self.helper.init_block(inputs, 'init_block')
#         print(concat.shape)
        bottleneck1 = self.helper.bottleneck_downsample(concat, 64)
        model = Model(inputs, bottleneck1)
        return model
    def compile_model():
        pass
    
    def train_model():
        pass
    
        

In [136]:
enet = ENet().build_model()

(None, 256, 256, 16)
(None, 128, 128, 64)


In [126]:
enet.summary()

Model: "functional_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_25 (ZeroPadding2 (None, 514, 514, 3)  0           input_42[0][0]                   
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 256, 256, 13) 364         zero_padding2d_25[0][0]          
__________________________________________________________________________________________________
max_pooling2d_40 (MaxPooling2D) (None, 256, 256, 3)  0           input_42[0][0]                   
______________________________________________________________________________________